# Before Building Model

### imports & variables & load 

In [ ]:
#imports
import numpy as np
import pandas as pd
import os,random

from xgboost import XGBClassifier

#variabels
TRAIN_PATH = "../input/tabular-playground-series-feb-2022/train.csv"
TEST_PATH = "../input/tabular-playground-series-feb-2022/test.csv"
SAMPLE_SUBMISSION_PATH = "../input/tabular-playground-series-feb-2022/sample_submission.csv"
SUBMISSION_PATH = "submission.csv"

ID = "row_id"
TARGET = "target"

SEED = 2022
def seed_everything(seed=SEED):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    
TREE_METHOD = 'gpu_hist'

#load
train = pd.read_csv(TRAIN_PATH)
test = pd.read_csv(TEST_PATH)

def reduce_memory_usage(df, verbose=True):
    numerics = ["int8", "int16", "int32", "int64", "float16", "float32", "float64"]
    start_mem = df.memory_usage().sum() / 1024 ** 2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if (
                    c_min > np.finfo(np.float16).min
                    and c_max < np.finfo(np.float16).max
                ):
                    df[col] = df[col].astype(np.float16)
                elif (
                    c_min > np.finfo(np.float32).min
                    and c_max < np.finfo(np.float32).max
                ):
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024 ** 2
    if verbose:
        print(
            "Mem. usage decreased to {:.2f} Mb ({:.1f}% reduction)".format(
                end_mem, 100 * (start_mem - end_mem) / start_mem
            )
        )
    return df

train = reduce_memory_usage(train)
test = reduce_memory_usage(test)

def getLabelCount(df,target):
    return [( labelValue,len(train.loc[df[target] == labelValue]) ) for labelValue in df[target].unique()]

labelCount = getLabelCount(train,TARGET)
labelCount

# Build model

In [ ]:
y = train[TARGET]
X = train.drop([ID,TARGET],axis=1)

model = XGBClassifier(tree_method=TREE_METHOD) 
model.fit(X, y)

# After Building Model

In [ ]:
X_test = test.drop([ID],axis=1)

submission = pd.read_csv(SAMPLE_SUBMISSION_PATH)
submission[TARGET] =model.predict(X_test)
submission.to_csv(SUBMISSION_PATH,index=False)
submission.head()